In [1]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import cv2

In [16]:
lines = []
with open('./data/data/driving_log1.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    sourcepath = line[0]
    filename = sourcepath.split('/')[-1]
    current_path = './data/data/IMG/'+filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*(-1.0))
        
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

In [18]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)
model.save('model.h5')

Train on 12857 samples, validate on 3215 samples
Epoch 1/5
12857/12857 [==============================] - 9s - loss: 2.5693 - val_loss: 4.5441
Epoch 2/5
12857/12857 [==============================] - 9s - loss: 4.7355 - val_loss: 2.4845
Epoch 3/5
12857/12857 [==============================] - 9s - loss: 3.7746 - val_loss: 1.4335
Epoch 4/5
12857/12857 [==============================] - 9s - loss: 3.1987 - val_loss: 2.2840
Epoch 5/5
12857/12857 [==============================] - 9s - loss: 3.5141 - val_loss: 3.3697


from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2)
model.save('model.h5')

In [7]:
nb_classes = 43

training_file = './train.p'
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
features, labels = train['features'], train['labels']
assert(len(features) == len(labels))

rand_state = np.random.randint(0, 100)
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=rand_state)

FileNotFoundError: [Errno 2] No such file or directory: './train.p'

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, [227, 227])

y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, nb_classes)

In [ ]:
# TODO: pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)

shape = (fc7.get_shape().as_list()[-1], nb_classes)  # use this shape for the weight matrix
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.add(tf.matmul(fc7, fc8W),fc8b)
probs = tf.nn.softmax(logits)

In [ ]:
# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.
rate = 0.001
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
BATCH_SIZE = 256
EPOCHS = 200

In [ ]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
# TODO: Train and evaluate the feature extraction model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './cnn_lenet/lenet')
    print("Model saved")